In [1]:
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm
import torch
import math
from gpt2.modeling_gpt2 import GPT2LMHeadModel
import numpy as np
from transformers import BertTokenizer, GPT2LMHeadModel

In [2]:
class CanaryDataset(Dataset):
    def __init__(self, canary, tokenizer, num_digit):
        self.canary = canary
        self.tokenizer = tokenizer
        if num_digit == 1:
            self.data = self.build_data_1()
        elif num_digit == 2:
            self.data = self.build_data_2()
        elif num_digit == 3:
            self.data = self.build_data_3()
        elif num_digit == 4:
            self.data = self.build_data_4()
        elif num_digit == 5:
            self.data = self.build_data_5()
        elif num_digit == 6:
            self.data = self.build_data_6()
    
    def build_data_1(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            text = f'我的单号是54168{i}'
            texts.append(text)
            encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))
    
    def build_data_2(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                text = f'我的单号是5416{i}{j}'
                texts.append(text)
                encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))

    def build_data_3(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                for k in range(10):
                    text = f'我的单号是541{i}{j}{k}'
                    texts.append(text)
                    encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))
    
    def build_data_4(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                for k in range(10):
                    for l in range(10):
                        text = f'我的单号是54{i}{j}{k}{l}'
                        texts.append(text)
                        encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))
    
    def build_data_5(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                for k in range(10):
                    for l in range(10):
                        for m in range(10):
                            text = f'我的单号是5{i}{j}{k}{l}{m}'
                            texts.append(text)
                            encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))
    
    def build_data_6(self):
        # '我的单号是541684'
        texts = []
        encoded_texts = []
        for i in tqdm(range(10)):
            for j in range(10):
                for k in range(10):
                    for l in range(10):
                        for m in range(10):
                            for n in range(10):
                                text = f'我的单号是{i}{j}{k}{l}{m}{n}'
                                texts.append(text)
                                encoded_texts.append(self.tokenizer.encode(text))
        assert self.canary in texts
        return list(zip(texts, encoded_texts))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        return self.data[index]

    def collate(self, unpacked_data):
        texts, encoded_texts = zip(*unpacked_data)
        max_length = max([len(encoded_text) for encoded_text in encoded_texts])
        padded_encoded_texts = [
            encoded_text + [self.tokenizer.pad_token_id] * (max_length - len(encoded_text))
            for encoded_text in encoded_texts
        ]
        return texts, torch.tensor(padded_encoded_texts)

def calculate_ppl(inputs, model):
    device = model.device
    input_ids = inputs.to(device)
    label_ids = input_ids.clone()

    with torch.no_grad():
        output = model(input_ids=input_ids, labels=label_ids)
        loss = output.loss
        sequence_lengths = len(input_ids)
        ppl_steps = torch.exp(loss / sequence_lengths)

    return ppl_steps.cpu().numpy()

def get_exposure(model, dataloader, CANARY, TOTAL_CANDIDATES):
    ppls = {}
    forward_pbar = tqdm(dataloader)
    forward_pbar.set_description('forward execution')
    for batch_input in forward_pbar:
        input_text, input_encoded_text = batch_input
        # batch_ppl = calculate_ppl(input_encoded_text, model)
        batch_ppl = [calculate_ppl(encoded_text, model) for encoded_text in input_encoded_text]
        ppls.update(dict(zip(input_text, batch_ppl)))
        forward_pbar.set_postfix({'ppl': '{:.7f}'.format(np.array(batch_ppl).mean())})

    sorted_ppls = {k: (i+1, v) for i, (k, v) in enumerate(sorted(ppls.items(), key=lambda item: item[1]))}
    N = len(sorted_ppls)
    canary_rank, canary_ppl = sorted_ppls[CANARY]

    canary_exposure = math.log(TOTAL_CANDIDATES, 2) - math.log(canary_rank, 2)

    print("canary ppl = ", canary_ppl)
    print("canary exposure = ", canary_exposure)
    print("canary ranking = ", canary_rank)

    return canary_exposure, canary_rank, canary_ppl

In [3]:
model_path = "..\\SDP_TrainOptim_CFT\\model\\best_model\\"
tok_path = '..\\..\\Raw_GPT2\\vocab.txt'
CANARY = '我的单号是541684'
tokenizer = BertTokenizer(vocab_file=tok_path)
model = GPT2LMHeadModel.from_pretrained(model_path)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('using device:{}'.format(device))
model.eval()
model.to(device)

using device:cuda


GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(21128, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dro

In [4]:
test_num = [1, 2, 3, 4, 5, 6]
for i in test_num:
    print('='*60)
    print('now testing generate ', i, ' number:')
    canary_corpus = CanaryDataset(CANARY, tokenizer, i)
    print('len(canary_corpus) = ', len(canary_corpus))
    dataloader = DataLoader(dataset=canary_corpus,
                            shuffle=False,
                            batch_size=16,
                            collate_fn=canary_corpus.collate)

    canary_exposure, canary_rank, canary_ppl = get_exposure(model, dataloader, CANARY, len(canary_corpus))
    print('='*60)

now testing generate  1  number:


100%|██████████| 10/10 [00:00<00:00, 4995.60it/s]


len(canary_corpus) =  10


forward execution: 100%|██████████| 1/1 [00:03<00:00,  3.73s/it, ppl=1.7220170]


canary ppl =  1.7171493
canary exposure =  1.7369655941662063
canary ranking =  3
now testing generate  2  number:


...

canary ppl =  1.7171493
canary exposure =  0.4883902789365777
canary ranking =  71282
now testing generate  6  number:


100%|██████████| 10/10 [02:11<00:00, 13.19s/it]


len(canary_corpus) =  1000000


forward execution: 100%|██████████| 62500/62500 [5:17:05<00:00,  3.29it/s, ppl=1.6879735]   


canary ppl =  1.7171493
canary exposure =  0.6032333035803283
canary ranking =  658277
